### CURSO YOUTUBE 

In [1]:
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem.porter import PorterStemmer
import ast

In [3]:
# carregando os dados 
credits_df = pd.read_csv('credits.csv')
movies_df = pd.read_csv('movies.csv')
credits_df

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [4]:
movies_df

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10-12-2009,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",19-05-2007,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",26-10-2015,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",16-07-2012,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",07-03-2012,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",04-09-1992,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],26-12-2011,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [5]:
movies_df = movies_df.merge(credits_df, on='title')

# pegando colunas importantes
movies_df = movies_df[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

# tirando valores nulos
movies_df.dropna(inplace=True)

In [6]:
# convertendo gêneros, palavras-chave, elenco e diretores em listas de nomes
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

def fetch_directors(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [7]:
# realizando as conversoes 
movies_df['genres'] = movies_df['genres'].apply(convert)
movies_df['keywords'] = movies_df['keywords'].apply(convert)
movies_df['cast'] = movies_df['cast'].apply(convert3)
movies_df['crew'] = movies_df['crew'].apply(fetch_directors)

# removendo espaçoes em branco
movies_df['overview'] = movies_df['overview'].apply(lambda x: x.split())
movies_df['genres'] = movies_df['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
movies_df['cast'] = movies_df['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
movies_df['crew'] = movies_df['crew'].apply(lambda x: [i.replace(" ", "") for i in x])

# criando a coluna tags 
movies_df['tags'] = movies_df['overview'] + movies_df['genres'] + movies_df['keywords'] + movies_df['cast'] + movies_df['crew']

In [8]:
df = movies_df[['movie_id', 'title', 'tags', 'genres']] # gerando o DataFrame com as colunas necessárias
df['tags'] = df['tags'].apply(lambda x: ' '.join(x)) # convertendo  lista para string
df['tags'] = df['tags'].apply(lambda x: x.lower()) # convertendo as tags para minúsculas

# inicializando o countvectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(df['tags']).toarray()

ps = PorterStemmer()

/tmp/ipykernel_34747/3630895600.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x: ' '.join(x)) # convertendo  lista para string
/tmp/ipykernel_34747/3630895600.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x: x.lower()) # convertendo as tags para minúsculas


In [9]:
# aplica stemming
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y)

# aplicando stemming
df['tags'] = df['tags'].apply(stem)

# calculando similaridade coseno
similarity = cosine_similarity(vectors)

/tmp/ipykernel_34747/2276503994.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(stem)


In [10]:
def recommend(movie): # função de recomendação 
    movie_index = df[df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    for i in movies_list:
        print(df.iloc[i[0]].title)

In [11]:
# testando 
recommend('Avatar') 

Titan A.E.
Independence Day
Small Soldiers
Aliens vs Predator: Requiem
Battle: Los Angeles


In [12]:
# testando 
recommend('Iron Man') 

Iron Man 2
Iron Man 3
Avengers: Age of Ultron
Captain America: Civil War
The Avengers


In [13]:
# testando 
recommend('Liar Liar')

13 Going on 30
Heartbreakers
A Simple Wish
A Good Year
21 & Over


In [14]:
# testando 
recommend('Captain America: Civil War')

Captain America: The First Avenger
Captain America: The Winter Soldier
Iron Man 3
Avengers: Age of Ultron
Iron Man 2


### CURSO UDEMY 

In [5]:
import pandas as pd

# carregando os dados 
r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, usecols=range(3))

m_cols = ['movie_id', 'title']
movies = pd.read_csv('u.item', sep='|', names=m_cols, usecols=range(2), encoding='ISO-8859-1')

ratings = pd.merge(movies, ratings)


ratings.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


In [6]:
# pegando as avaliaçoes dos filmes 
movieRatings = ratings.pivot_table(index=['user_id'], columns=['title'], values=['rating'])

In [7]:

movieRatings.columns = movieRatings.columns.get_level_values(1)
starWarsRatings = movieRatings['Star Wars (1977)'] # avaliação do star wars 
starWarsRatings

user_id
0      5.0
1      5.0
2      5.0
3      NaN
4      5.0
      ... 
939    NaN
940    4.0
941    NaN
942    5.0
943    4.0
Name: Star Wars (1977), Length: 944, dtype: float64

In [8]:

similarMovies = movieRatings.corrwith(starWarsRatings) # analisa a similaridade entre as avaliações de star wars com outros filmes
similarMovies = similarMovies.dropna()
df = pd.DataFrame(similarMovies)
df.head(10)

/home/williansoder/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/williansoder/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/williansoder/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/williansoder/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/home/williansoder/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


,0
title,
'Til There Was You (1997),0.872872
1-900 (1994),-0.645497
101 Dalmatians (1996),0.211132
12 Angry Men (1957),0.184289
187 (1997),0.027398
2 Days in the Valley (1996),0.066654
"20,000 Leagues Under the Sea (1954)",0.289768
2001: A Space Odyssey (1968),0.230884
"39 Steps, The (1935)",0.106453


In [9]:
similarMovies.sort_values(ascending=False) # ordena os filmes 

title
Hollow Reed (1996)            1.0
Commandments (1997)           1.0
Cosi (1996)                   1.0
No Escape (1994)              1.0
Stripes (1981)                1.0
                             ... 
For Ever Mozart (1996)       -1.0
Frankie Starlight (1995)     -1.0
I Like It Like That (1994)   -1.0
American Dream (1990)        -1.0
Theodore Rex (1995)          -1.0
Length: 1410, dtype: float64

In [10]:
import numpy as np

# agrupando as linhas das avaliações em uma só com a media  
movieStats = ratings.groupby('title').agg({'rating': [np.size, np.mean]})

/tmp/ipykernel_36754/3776584072.py:3: FutureWarning: The provided callable <function mean at 0x74798c0e5630> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movieStats = ratings.groupby('title').agg({'rating': [np.size, np.mean]})


In [11]:
popularMovies = movieStats['rating']['size'] >= 100 # pegando os filmes melhores avaliados 
movieStats[popularMovies].sort_values([('rating', 'mean')], ascending=False)[:15]

rating          
                                         size      mean
title                                                  
Close Shave, A (1995)                     112  4.491071
Schindler's List (1993)                   298  4.466443
Wrong Trousers, The (1993)                118  4.466102
Casablanca (1942)                         243  4.456790
Shawshank Redemption, The (1994)          283  4.445230
Rear Window (1954)                        209  4.387560
Usual Suspects, The (1995)                267  4.385768
Star Wars (1977)                          584  4.359589
12 Angry Men (1957)                       125  4.344000
Citizen Kane (1941)                       198  4.292929
To Kill a Mockingbird (1962)              219  4.292237
One Flew Over the Cuckoo's Nest (1975)    264  4.291667
Silence of the Lambs, The (1991)          390  4.289744
North by Northwest (1959)                 179  4.284916
Godfather, The (1972)                     413  4.283293

In [ ]:
df = movieStats[popularMovies].join(pd.DataFrame(similarMovies, columns=['similarity']), how = 'right')
df.head()

In [14]:
import pandas as pd

# carregando os dados 
r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, usecols=range(3))

m_cols = ['movie_id', 'title']
movies = pd.read_csv('u.item', sep='|', names=m_cols, usecols=range(2), encoding='ISO-8859-1')

ratings = pd.merge(movies, ratings)
ratings.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


In [15]:
# pegando as avaliações 
userRatings = ratings.pivot_table(index=['user_id'], columns=['title'], values=['rating'])
userRatings.head()

rating                                     \
title   'Til There Was You (1997) 1-900 (1994) 101 Dalmatians (1996)   
user_id                                                                
0                             NaN          NaN                   NaN   
1                             NaN          NaN                   2.0   
2                             NaN          NaN                   NaN   
3                             NaN          NaN                   NaN   
4                             NaN          NaN                   NaN   

                                                                    \
title   12 Angry Men (1957) 187 (1997) 2 Days in the Valley (1996)   
user_id                                                              
0                       NaN        NaN                         NaN   
1                       5.0        NaN                         NaN   
2                       NaN        NaN                         NaN   
3                       NaN        2.0                         NaN   
4                       NaN        NaN                         NaN   

                                                                          \
title   20,000 Leagues Under the Sea (1954) 2001: A Space Odyssey (1968)   
user_id                                                                    
0                                       NaN                          NaN   
1                                       3.0                          4.0   
2                                       NaN                          NaN   
3                                       NaN                          NaN   
4                                       NaN                          NaN   

                                                                          ...  \
title   3 Ninjas: High Noon At Mega Mountain (1998) 39 Steps, The (1935)  ...   
user_id                                                                   ...   
0                                               NaN                  NaN  ...   
1                                               NaN                  NaN  ...   
2                                               1.0                  NaN  ...   
3                                               NaN                  NaN  ...   
4                                               NaN                  NaN  ...   

                                                                         \
title   Yankee Zulu (1994) Year of the Horse (1997) You So Crazy (1994)   
user_id                                                                   
0                      NaN                      NaN                 NaN   
1                      NaN                      NaN                 NaN   
2                      NaN                      NaN                 NaN   
3                      NaN                      NaN                 NaN   
4                      NaN                      NaN                 NaN   

                                                                          \
title   Young Frankenstein (1974) Young Guns (1988) Young Guns II (1990)   
user_id                                                                    
0                             NaN               NaN                  NaN   
1                             5.0               3.0                  NaN   
2                             NaN               NaN                  NaN   
3                             NaN               NaN                  NaN   
4                             NaN               NaN                  NaN   

                                                                               \
title   Young Poisoner's Handbook, The (1995) Zeus and Roxanne (1997) unknown   
user_id                                                                         
0                                         NaN                     NaN     NaN   
1                                         NaN                     NaN     4.0   
2               

In [16]:
corrMatrix = userRatings.corr() # relação entre a avaliação de dois filmes
corrMatrix.head()

rating               \
title                            'Til There Was You (1997) 1-900 (1994)   
       title                                                              
rating 'Til There Was You (1997)                       1.0          NaN   
       1-900 (1994)                                    NaN          1.0   
       101 Dalmatians (1996)                          -1.0          NaN   
       12 Angry Men (1957)                            -0.5          NaN   
       187 (1997)                                     -0.5          NaN   

                                                                            \
title                            101 Dalmatians (1996) 12 Angry Men (1957)   
       title                                                                 
rating 'Til There Was You (1997)             -1.000000           -0.500000   
       1-900 (1994)                                NaN                 NaN   
       101 Dalmatians (1996)                  1.000000           -0.049890   
       12 Angry Men (1957)                   -0.049890            1.000000   
       187 (1997)                             0.269191            0.666667   

                                                                         \
title                            187 (1997) 2 Days in the Valley (1996)   
       title                                                              
rating 'Til There Was You (1997)  -0.500000                    0.522233   
       1-900 (1994)                     NaN                         NaN   
       101 Dalmatians (1996)       0.269191                    0.048973   
       12 Angry Men (1957)         0.666667                    0.256625   
       187 (1997)                  1.000000                    0.596644   

                                                                      \
title                            20,000 Leagues Under the Sea (1954)   
       title                                                           
rating 'Til There Was You (1997)                                 NaN   
       1-900 (1994)                                              NaN   
       101 Dalmatians (1996)                                0.266928   
       12 Angry Men (1957)                                  0.274772   
       187 (1997)                                                NaN   

                                                               \
title                            2001: A Space Odyssey (1968)   
       title                                                    
rating 'Til There Was You (1997)                    -0.426401   
       1-900 (1994)                                 -0.981981   
       101 Dalmatians (1996)                        -0.043407   
       12 Angry Men (1957)                           0.178848   
       187 (1997)                                   -0.554700   

                                                                              \
title                            3 Ninjas: High Noon At Mega Mountain (1998)   
       title                                                                   
rating 'Til There Was You (1997)                                         NaN   
       1-900 (1994)                                                      NaN   
       101 Dalmatians (1996)                                             NaN   
       12 Angry Men (1957)                                               NaN   
       187 (1997)                                                        NaN   

                                                       ...                     \
title                            39 Steps, The (1935)  ... Yankee Zulu (1994)   
       title                                           ...                      
rating 'Til There Was You (1997)                  NaN  ...                NaN   
       1-900 (1994)                               NaN  ...                NaN   
       101 Dalmatians (1996)                 0.111111  ...                NaN   


In [17]:
# relação entre a avaliação (usando 100 pessoas)
corrMatrix = userRatings.corr(method='pearson', min_periods=100)
corrMatrix.head()

rating               \
title                            'Til There Was You (1997) 1-900 (1994)   
       title                                                              
rating 'Til There Was You (1997)                       NaN          NaN   
       1-900 (1994)                                    NaN          NaN   
       101 Dalmatians (1996)                           NaN          NaN   
       12 Angry Men (1957)                             NaN          NaN   
       187 (1997)                                      NaN          NaN   

                                                                            \
title                            101 Dalmatians (1996) 12 Angry Men (1957)   
       title                                                                 
rating 'Til There Was You (1997)                   NaN                 NaN   
       1-900 (1994)                                NaN                 NaN   
       101 Dalmatians (1996)                       1.0                 NaN   
       12 Angry Men (1957)                         NaN                 1.0   
       187 (1997)                                  NaN                 NaN   

                                                                         \
title                            187 (1997) 2 Days in the Valley (1996)   
       title                                                              
rating 'Til There Was You (1997)        NaN                         NaN   
       1-900 (1994)                     NaN                         NaN   
       101 Dalmatians (1996)            NaN                         NaN   
       12 Angry Men (1957)              NaN                         NaN   
       187 (1997)                       NaN                         NaN   

                                                                      \
title                            20,000 Leagues Under the Sea (1954)   
       title                                                           
rating 'Til There Was You (1997)                                 NaN   
       1-900 (1994)                                              NaN   
       101 Dalmatians (1996)                                     NaN   
       12 Angry Men (1957)                                       NaN   
       187 (1997)                                                NaN   

                                                               \
title                            2001: A Space Odyssey (1968)   
       title                                                    
rating 'Til There Was You (1997)                          NaN   
       1-900 (1994)                                       NaN   
       101 Dalmatians (1996)                              NaN   
       12 Angry Men (1957)                                NaN   
       187 (1997)                                         NaN   

                                                                              \
title                            3 Ninjas: High Noon At Mega Mountain (1998)   
       title                                                                   
rating 'Til There Was You (1997)                                         NaN   
       1-900 (1994)                                                      NaN   
       101 Dalmatians (1996)                                             NaN   
       12 Angry Men (1957)                                               NaN   
       187 (1997)                                                        NaN   

                                                       ...                     \
title                            39 Steps, The (1935)  ... Yankee Zulu (1994)   
       title                                           ...                      
rating 'Til There Was You (1997)                  NaN  ...                NaN   
       1-900 (1994)                               NaN  ...                NaN   
       101 Dalmatians (1996)                      NaN  ...                NaN   


In [18]:
myRatings = userRatings.loc[0].dropna() # tirando valores nulos 
myRatings

        title                          
rating  Empire Strikes Back, The (1980)    5.0
        Gone with the Wind (1939)          1.0
        Star Wars (1977)                   5.0
Name: 0, dtype: float64

In [20]:
simCandidates = pd.Series() # criando uma serie 

# somar filmes similares aos classificados 
for i in range(0, len(myRatings.index)):
    print('Adding sims for ' + str(myRatings.index[i]) + '...')
    sims = corrMatrix[myRatings.index[i]].dropna()
    sims = sims.map(lambda x: x * myRatings[1])
    simCandidates = simCandidates._append(sims)

print('Sorting...')
# ordenando 
simCandidates.sort_values(inplace=True, ascending=False)
print(simCandidates.head(10)) # analisando sua similaridade e mostrando os 10 mais semelhantes

Adding sims for ('rating', 'Empire Strikes Back, The (1980)')...
Adding sims for ('rating', 'Gone with the Wind (1939)')...
Adding sims for ('rating', 'Star Wars (1977)')...
Sorting...
(rating, Empire Strikes Back, The (1980))    1.000000
(rating, Star Wars (1977))                   1.000000
(rating, Gone with the Wind (1939))          1.000000
(rating, Star Wars (1977))                   0.748353
(rating, Empire Strikes Back, The (1980))    0.748353
(rating, Return of the Jedi (1983))          0.721229
(rating, Return of the Jedi (1983))          0.672556
(rating, Raiders of the Lost Ark (1981))     0.538659
(rating, Raiders of the Lost Ark (1981))     0.536117
(rating, Wizard of Oz, The (1939))           0.430219
dtype: float64


/tmp/ipykernel_36754/3309020545.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sims = sims.map(lambda x: x * myRatings[1])
/tmp/ipykernel_36754/3309020545.py:6: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  simCandidates = simCandidates._append(sims)
/tmp/ipykernel_36754/3309020545.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sims = sims.map(lambda x: x * myRatings[1])
/tmp/ipykernel_36754

In [21]:
simCandidates = simCandidates.groupby(simCandidates.index).sum() # agrupa todas as iterações do filme na matriz e a soma
simCandidates.sort_values(inplace=True, ascending=False)
simCandidates.head(10)

(rating, Empire Strikes Back, The (1980))              1.884040
(rating, Star Wars (1977))                             1.877560
(rating, Return of the Jedi (1983))                    1.603032
(rating, Gone with the Wind (1939))                    1.264895
(rating, Raiders of the Lost Ark (1981))               1.220594
(rating, Wizard of Oz, The (1939))                     0.984229
(rating, E.T. the Extra-Terrestrial (1982))            0.883620
(rating, Back to the Future (1985))                    0.877443
(rating, Indiana Jones and the Last Crusade (1989))    0.834655
(rating, Casablanca (1942))                            0.809604
dtype: float64

In [22]:
filteredSims = simCandidates.drop(myRatings.index)
filteredSims

(rating, Return of the Jedi (1983))            1.603032
(rating, Raiders of the Lost Ark (1981))       1.220594
(rating, Wizard of Oz, The (1939))             0.984229
(rating, E.T. the Extra-Terrestrial (1982))    0.883620
(rating, Back to the Future (1985))            0.877443
                                                 ...   
(rating, Annie Hall (1977))                   -0.102355
(rating, Real Genius (1985))                  -0.110574
(rating, Remains of the Day, The (1993))      -0.112067
(rating, This Is Spinal Tap (1984))           -0.127295
(rating, First Wives Club, The (1996))        -0.194496
Length: 265, dtype: float64